# Real-time monitoring and logging of data from optical distance sensor

The following class encapsulates the measurement for the [Leuze ODSL 9/D26-450-S12 sensor](http://www.leuze.com/en/deutschland/produkte/messende_sensoren/abstandssensoren/optische_abstandssensoren/odsl_10/odsl_13/selector.php?supplier_aid=50111159&grp_id=A1-2-1-1-2-1&lang=eng) via serial port.

In [1]:
from datetime import datetime
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg
import numpy as np
import collections
import serial

class LaserSensor():

    def __init__(self, port, baudrate, write_csv=False):        
        self.app = QtGui.QApplication([])
        self.write_csv = write_csv
        self.connect_serial(port, baudrate) # open serial port

    def __del__(self):
        if (self.write_csv):
            self.file_csv.close()
        self.ser.close() # close port

    def connect_serial(self, port, baudrate):
        self.ser = serial.Serial()
        self.ser.port = port
        self.ser.baudrate = baudrate
        self.ser.open()

        while (True): # skip the first data
            if (self.get_data()):
                break

    def get_data(self):
        '''
        See http://goo.gl/PMwlfv for technical description of the sensor
        '''
        eol = '\r' # Read a '\r' terminated line
        leneol = len(eol)
        line = bytearray()
        while True:
            c = self.ser.read(1) # read one byte
            if c:
                line += c
                if line[-leneol:] == eol:
                    break
            else:
                break
        return line

    def measure(self):
        '''
        Start monitoring and logging
        '''
        self.set_plot()
        self.set_timer()

        if (self.write_csv):
            self.time_now = self.time_pre = datetime.now()
            file_name = self.ser.port+self.time_now.strftime('_%Y%m%d%H%M%S')+'.csv'
            self.file_csv = open(file_name,'a')
    
        self.app.exec_()

    def set_plot(self, sample_interval=0.1, time_window=10.):
        self.sample_interval = int(sample_interval*1000)
        self.buffer_size = int(time_window/sample_interval)
        self.data_buffer = collections.deque([0.0]*self.buffer_size, self.buffer_size)
        self.x = np.linspace(-time_window, 0.0, self.buffer_size)
        self.y = np.zeros(self.buffer_size, dtype=np.float)

        self.plt = pg.plot(title='Signal Monitoring')
        self.plt.resize(800, 400)
        self.plt.showGrid(x=True, y=True)
        self.plt.setLabel('left','Distance','m')
        self.plt.setLabel('bottom','Time','s')
        self.curve = self.plt.plot(self.x, self.y, pen=(160,160,160))

    def update_plot(self):
        data = float(self.get_data())/10000.0
        self.data_buffer.append(data)
        self.y[:] = self.data_buffer
        self.curve.setData(self.x, self.y)
        self.app.processEvents()

        if (self.write_csv):
            self.time_now = datetime.now()
            if (self.time_now > self.time_pre):
                csv = '{0},{1}\n'.format(str(self.time_now)[:-3], data)
                self.file_csv.write(csv)
                self.time_pre = self.time_now

    def set_timer(self):
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update_plot)
        self.timer.start(self.sample_interval)

### Testing

In [ ]:
sensor = LaserSensor('COM9', 9600)
sensor.measure()

![Plot](figs/laser_sensor.png)